In [10]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  # optional if you want a quick, cleaner boxplot


### Citations:
https://eccc-msc.github.io/open-data/msc-geomet/ogc_api_en/

In [12]:
# API endpoint
url = "https://api.weather.gc.ca/collections/climate-daily/items"
# Query parameters
params = {
    "datetime": "2019-01-01/2025-03-31",
    "limit": 10000,  # you can paginate if needed
    "PROVINCE_CODE": "BC",
}
# Make request
response = requests.get(url, params=params)
data = response.json()

In [3]:
# Convert to DataFrame
features = data["features"]
records = [f["properties"] for f in features]
climate_df = pd.DataFrame(records)

In [4]:
climate_df.size

340000

In [5]:
# Preview
display(climate_df.head())

,MIN_TEMPERATURE_FLAG,TOTAL_SNOW,PROVINCE_CODE,MAX_TEMPERATURE_FLAG,STATION_NAME,ID,MAX_REL_HUMIDITY,MAX_TEMPERATURE,MAX_REL_HUMIDITY_FLAG,LOCAL_YEAR,...,SNOW_ON_GROUND,TOTAL_PRECIPITATION_FLAG,SNOW_ON_GROUND_FLAG,TOTAL_RAIN,TOTAL_SNOW_FLAG,TOTAL_RAIN_FLAG,COOLING_DEGREE_DAYS_FLAG,COOLING_DEGREE_DAYS,DIRECTION_MAX_GUST,HEATING_DEGREE_DAYS_FLAG
0,None,NaN,ON,None,UPSALA (AUT),6049095.2019.1.20,74.0,-20.4,None,2019,...,40.0,None,None,NaN,None,None,None,0.0,NaN,None
1,None,NaN,ON,None,UPSALA (AUT),6049095.2019.1.9,83.0,-13.6,None,2019,...,41.0,None,None,NaN,None,None,None,0.0,33.0,None
2,None,NaN,ON,None,UPSALA (AUT),6049095.2019.1.17,79.0,-14.4,None,2019,...,41.0,None,None,NaN,None,None,None,0.0,NaN,None
3,None,NaN,ON,None,UPSALA (AUT),6049095.2019.1.21,83.0,-11.8,None,2019,...,40.0,None,None,NaN,None,None,None,0.0,NaN,None
4,None,NaN,ON,None,UPSALA (AUT),6049095.2019.1.27,72.0,-24.7,None,2019,...,46.0,None,None,NaN,None,None,None,0.0,NaN,None


In [6]:
#writeback to csv => aggregation 
climate_df.to_csv("data/climate_data/monthly_climate_data.csv")

In [7]:
climate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   MIN_TEMPERATURE_FLAG      43 non-null     object 
 1   TOTAL_SNOW                6093 non-null   float64
 2   PROVINCE_CODE             10000 non-null  object 
 3   MAX_TEMPERATURE_FLAG      43 non-null     object 
 4   STATION_NAME              10000 non-null  object 
 5   ID                        10000 non-null  object 
 6   MAX_REL_HUMIDITY          3812 non-null   float64
 7   MAX_TEMPERATURE           9955 non-null   float64
 8   MAX_REL_HUMIDITY_FLAG     70 non-null     object 
 9   LOCAL_YEAR                10000 non-null  int64  
 10  LOCAL_DAY                 10000 non-null  int64  
 11  SPEED_MAX_GUST            2505 non-null   float64
 12  DIRECTION_MAX_GUST_FLAG   44 non-null     object 
 13  CLIMATE_IDENTIFIER        10000 non-null  object 
 14  MEAN_TE